In [1]:
import os
import sys
import numpy as np
import cPickle
import random
import math
import h5py
from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences



np.random.seed(1)
random.seed(1)

Using TensorFlow backend.


In [16]:
# Initialize parameters
DATASET = 'cora'

embedding_size = 50
learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 3
path_size = 10

g_batch_size = 200
g_learning_rate = 0.01
g_sample_size = 100

use_feature = True
update_emb = True
layer_loss =  True

In [33]:
# Load the dataset
NAMES = ['x', 'y', 'tx', 'ty', 'graph']
OBJECTS = []
for i in range(len(NAMES)):
    OBJECTS.append(cPickle.load(open("/hdd2/graph_embedding/planetoid/data/trans.{}.{}".format(DATASET, NAMES[i]))))
x, y, tx, ty, graph = tuple(OBJECTS)

In [34]:
def comp_iter(iter):
    """an auxiliary function used for computing the number of iterations given the argument iter.
    iter can either be an int or a float.
    """
    if iter >= 1:
        return iter
    return 1 if random.random() < iter else 0

In [35]:
# Sample a collections of paths from the graph

In [36]:
def initialize_parameters(l_emd_f_W_size, l_x_hid_W_size, l_y_W_size):
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]??
                        W2 : [2, 2, 8, 16]?
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)
    
    l_emd_f_W = tf.Variable(tf.truncated_normal(l_emd_f_W_size,
                            stddev=1.0 / math.sqrt(l_emd_f_W_size[1])))
    l_emd_f_b = tf.Variable(tf.zeros([l_emd_f_W_size[0]]))
    
    l_x_hid_W = tf.get_variable('l_x_hid_W', shape = l_x_hid_W_size,
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_x_hid_b = tf.get_variable('l_x_hid_b', shape = [l_x_hid_W_size[0], 1],
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_W = tf.get_variable('l_y_W', shape = l_y_W_size,
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_b = tf.get_variable('l_y_b', shape = [l_y_W_size[0], 1],
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    
    parameters = {'l_emd_f_W': l_emd_f_W,
                  'l_emd_f_b': l_emd_f_b,
                  'l_x_hid_W': l_x_hid_W,
                  'l_x_hid_b': l_x_hid_b,
                  'l_y_W': l_y_W,
                  'l_y_b': l_y_b}
    
    return parameters

In [37]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters([3,3], [3,4], [4,2])
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print(str(parameters['l_emd_f_W'].eval()))


[[-0.38732192  0.12907903  0.46030581]
 [ 0.05552411  0.5345757   1.12406611]
 [ 0.40012899  0.12259372 -0.88986874]]


In [38]:
def create_placeholders():
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    
    x_sym = tf.placeholder(tf.float32, shape = [None, x.shape[1]], name = 'x')
    y_sym = tf.placeholder(tf.int32, shape = [None, y.shape[1]], name = 'y')
    g_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'g')
    gy_sym = tf.placeholder(tf.int32, shape = [None, 1], name = 'gy')
    ind_sym = tf.placeholder(tf.int32, shape = [None], name = 'ind')
    
    
    return x_sym, y_sym, g_sym, gy_sym, ind_sym

In [39]:
# x_sym, y_sym, g_sym, gy_sym, ind_sym = create_placeholders()
# embeddings = tf.Variable(
#     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
# softmax_weights = tf.Variable(
#     tf.truncated_normal([vocabulary_size, embedding_size],
#                         stddev=1.0 / math.sqrt(embedding_size)))
# softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
# l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

In [40]:
# def build():
"""
Builds the model.
"""
np.random.seed(1)
random.seed(1)
tf.random_normal_initializer(seed = 1)
num_sampled = 64
num_ver = max(graph.keys()) + 1
vocabulary_size = num_ver

# graph = tf.Graph()

# with graph.as_default(), tf.device('/cpu:0'):
    
x_sym, y_sym, g_sym, gy_sym, ind_sym = create_placeholders()



# word embedding
tf.random_normal_initializer(seed = 1)
tf.set_random_seed(1)
embeddings = tf.Variable(
    tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
softmax_weights = tf.Variable(
    tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

##

l_x_hid = tf.layers.dense(inputs = x_sym, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
if use_feature:
    l_emd_z = tf.layers.dense(inputs = l_emd_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
    l_f = tf.concat([l_x_hid, l_emd_z], axis = 1)
    l_y = tf.layers.dense(inputs = l_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
else:
    l_y = tf.layers.dense(inputs = l_emd_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())

l_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_y, labels = y_sym))

if layer_loss and use_feature:
    l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_x_hid, labels = y_sym))
    l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_emd_z, labels = y_sym))

# if neg_samp == 0:
#     pass
# else:
g_loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, 
                               inputs = l_emd_f, labels = gy_sym, 
                               num_sampled = vocabulary_size, 
                               num_classes = vocabulary_size))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l_loss)

g_optimizer = tf.train.GradientDescentOptimizer(g_learning_rate).minimize(g_loss)

In [41]:
def gen_train_inst():
    """generator for batches for classification loss.
    """
    np.random.seed(1)
    random.seed(1)
    while True:
        ind = np.array(np.random.permutation(x.shape[0]), dtype = np.int32)
        i = 0
        while i < ind.shape[0]:
            j = min(ind.shape[0], i + batch_size)
            yield x[ind[i: j]], y[ind[i: j]], ind[i: j]
            i = j

In [42]:
def gen_label_graph():
    """generator for batches for label context loss.
    """
    np.random.seed(1)
    random.seed(1)
    labels, label2inst, not_label = [], dd(list), dd(list)
    for i in range(x.shape[0]):
        flag = False
        for j in range(y.shape[1]):
            if y[i, j] == 1 and not flag:
                labels.append(j)
                label2inst[j].append(i)
                flag = True
            elif y[i, j] == 0:
                not_label[j].append(i)

    while True:
        g, gy = [], []
        for _ in range(g_sample_size):
            x1 = random.randint(0, x.shape[0] - 1)
            label = labels[x1]
            if len(label2inst) == 1: continue
            x2 = random.choice(label2inst[label])
            g.append([x1, x2])
            gy.append(1.0)
#             for _ in range(neg_samp):
#                 g.append([x1, random.choice(not_label[label])])
#                 gy.append( - 1.0)
        yield np.array(g, dtype = np.int32), np.array(gy, dtype = np.float32)

In [43]:
def gen_graph():
    """generator for batches for graph context loss.
    """
    np.random.seed(1)
    random.seed(1)
    num_ver = max(graph.keys()) + 1

    while True:
        ind = np.random.permutation(num_ver)
        i = 0
        while i < ind.shape[0]:
            g, gy = [], []
            j = min(ind.shape[0], i + g_batch_size)
            for k in ind[i: j]:
                if len(graph[k]) == 0: continue
                path = [k]
                for _ in range(path_size):
                    path.append(random.choice(graph[path[-1]]))
                for l in range(len(path)):
                    for m in range(l - window_size, l + window_size + 1):
                        if m < 0 or m >= len(path): continue
                        g.append([path[l], path[m]])
                        gy.append(1.0)
#                         for _ in range(neg_samp):
#                             # if the random number euqals to path[m], the it creates noise!
#                             g.append([path[l], random.randint(0, num_ver - 1)])
#                             gy.append(- 1.0)
            yield np.array(g, dtype = np.int32), np.array(gy, dtype = np.float32)
            i = j

In [44]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [45]:
max_iter = 1
iter_graph = 0
iter_inst = 1
iter_label = 0

In [46]:
init_iter_label = 2000
init_iter_graph = 70
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

# init_train
with tf.Session() as session:
    tf.global_variables_initializer().run()
    print('init_iter_label')
    for i in range(init_iter_label):
        gx, gy = next(label_generator)
#         print(gx)
        feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
        _, l, res_l_emd_f = session.run([g_optimizer, g_loss, softmax_weights], feed_dict=feed_dict)
        print 'iter label', i, l
#         print(res_l_emd_f)
#         sys.exit(0)

#     sys.exit(0)
    print('init_iter_graph')
    for i in range(init_iter_graph):
        gx, gy = next(graph_generator)
        feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
        _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
        print 'iter graph', i, l
        
#     print('init_iter_label')
#     for i in range(init_iter_label):
#         gx, gy = next(label_generator)
#         feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#         _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#         print 'iter label', i, l

    iter_cnt = 0
    while True:
        for _ in range(max_iter):
            for _ in range(comp_iter(iter_graph)):
                gx, gy = next(graph_generator)
                feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
                _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)

            for _ in range(comp_iter(iter_inst)):
                xs, ys, indexs = next(inst_generator)
#                 gx, gy = next(label_generator)
                xs = xs.toarray()
                feed_dict={x_sym: xs, y_sym: ys, g_sym: indexs, gy_sym: indexs.reshape(indexs.shape[0], 1)}
                _, l = session.run([optimizer, l_loss], feed_dict=feed_dict)
    #           print 'iter inst', i, l

            for _ in range(comp_iter(iter_label)):
                gx, gy = next(label_generator)
                feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
                _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)


        predict_y = tf.arg_max(l_y, 1)
        correct_prediction = tf.equal(predict_y, tf.arg_max(y_sym, 1))

        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    #     print(accuracy)
#         print('number of iteration: %d', iter_cnt)
        iter_cnt += 1
        train_accuracy = accuracy.eval({x_sym: xs, y_sym: ys, g_sym: indexs, gy_sym: indexs.reshape(indexs.shape[0], 1)})
#         print('Train Accuracy: %.4f', train_accuracy)
        txs = tx.toarray()
        t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)
        test_accuracy = accuracy.eval({x_sym: txs, y_sym: ty, g_sym: t_index, gy_sym: t_index.reshape(t_index.shape[0], 1)})
        print('iter: %d, Tst Acc: %.4f, Trn Acc: %.4f' %(iter_cnt, test_accuracy, train_accuracy))
    

init_iter_label
iter label 0 7.90472
iter label 1 7.90459
iter label 2 7.90431
iter label 3 7.90408
iter label 4 7.90366
iter label 5 7.90378
iter label 6 7.90399
iter label 7 7.9035
iter label 8 7.90453
iter label 9 7.90344
iter label 10 7.90393
iter label 11 7.9056
iter label 12 7.90317
iter label 13 7.90332
iter label 14 7.90352
iter label 15 7.904
iter label 16 7.90335
iter label 17 7.90314
iter label 18 7.90284
iter label 19 7.90305
iter label 20 7.9026
iter label 21 7.90318
iter label 22 7.90249
iter label 23 7.90289
iter label 24 7.90246
iter label 25 7.90264
iter label 26 7.90221
iter label 27 7.90242
iter label 28 7.90217
iter label 29 7.90237
iter label 30 7.90182
iter label 31 7.90206
iter label 32 7.90207
iter label 33 7.90218
iter label 34 7.90197
iter label 35 7.90158
iter label 36 7.90247
iter label 37 7.90216
iter label 38 7.90145
iter label 39 7.90184
iter label 40 7.90142
iter label 41 7.90124
iter label 42 7.90245
iter label 43 7.90164
iter label 44 7.90114
iter labe

KeyboardInterrupt: 

In [ ]:
with tf.Session() as session:
    gx, gy = next(label_generator)
    tf.global_variables_initializer().run()
    feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
    _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
    print 'iter label', i, loss

In [271]:
indexs.shape

(120,)

In [243]:
xs, ys, indexs = next(inst_generator)
xs = xs.toarray()

In [246]:
print(type(xs))
print(type(ys))
print(type(indexs))
print(xs.shape)
print(ys.shape)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(120, 3703)
(120, 6)


In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [319]:
xs[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [320]:
txs[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [321]:
ys[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [324]:
t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)

In [431]:
res_l_emd_f.shape

(100, 50)